In [1]:
import os


In [2]:
%pwd

'c:\\Users\\aryaa\\OneDrive\\Desktop\\End-to-end-nlp\\NLP_project\\research'

In [3]:
os.chdir("../")

In [4]:
%pwd

'c:\\Users\\aryaa\\OneDrive\\Desktop\\End-to-end-nlp\\NLP_project'

In [5]:
from dataclasses import dataclass   
from pathlib import Path

@dataclass(frozen=True)
class DataIngestionConfig:
    root_dir: Path
    soure_URL: str
    local_data_file: Path
    unzip_dir: Path

In [6]:
from text_summarizer.constants import *
from text_summarizer.utils.common import read_yaml, create_directories

In [8]:
class ConfigurationManager:
    def __init__(
            self,
            config_path = CONFIG_FILE_PATH  ,
            params_filepath = PARAMS_FILE_PATH):
        self.config = read_yaml(config_path)
        self.params = read_yaml(params_filepath)
        create_directories([self.config.artifacts_root])
    
    def get_data_ingestion_config(self) -> DataIngestionConfig:
        config = self.config.data_ingestion
        create_directories([config.root_dir])

        data_ingestion_config= DataIngestionConfig(
            root_dir = config.root_dir,
            soure_URL=  config.source_URL,
            local_data_file=config.local_data_file,
            unzip_dir=config.unzip_dir
        )
        return data_ingestion_config    
            

In [9]:
import os
import urllib.request as request
import zipfile
from text_summarizer.logging import logger
from text_summarizer.utils.common import get_size

In [11]:
class DataIngestion:
    def __init__(self,config:DataIngestionConfig):
        self.config = config
    
    def download_file(self):
        if not os.path.exists(self.config.local_data_file):
            filename, headers = request.urlretrieve(
                url=self.config.soure_URL,
                filename=self.config.local_data_file
            )
            logger.info(f"{filename} download! with following info: \n{headers}")
        else:
            logger.info(f"File already exists of size: {get_size(Path(self.config.local_data_file))}")
    
    def extract_zip_file(self):
        unzip_path = self.config.unzip_dir
        os.makedirs(unzip_path,exist_ok=True)
        with zipfile.ZipFile(self.config.local_data_file,'r') as zip_ref:
            zip_ref.extractall(unzip_path)

In [13]:
try:
    config = ConfigurationManager()
    data_ingestion_config = config.get_data_ingestion_config()
    data_ingestion = DataIngestion(config=data_ingestion_config)
    data_ingestion.download_file()
    data_ingestion.extract_zip_file()
except Exception as e:
    raise e

[2024-05-07 21:16:33,196:INFO:common:yaml file: config\config.yaml loaded successfully]
[2024-05-07 21:16:33,210:INFO:common:yaml file: params.yaml loaded successfully]
[2024-05-07 21:16:33,212:INFO:common:created directory at: artifacts]
[2024-05-07 21:16:33,214:INFO:common:created directory at: artifacts/data_ingestion]


[2024-05-07 21:16:42,472:INFO:1229135680:artifacts/data_ingestion/data.zip download! with following info: 
Connection: close
Content-Length: 7903594
Cache-Control: max-age=300
Content-Security-Policy: default-src 'none'; style-src 'unsafe-inline'; sandbox
Content-Type: application/zip
ETag: "dbc016a060da18070593b83afff580c9b300f0b6ea4147a7988433e04df246ca"
Strict-Transport-Security: max-age=31536000
X-Content-Type-Options: nosniff
X-Frame-Options: deny
X-XSS-Protection: 1; mode=block
X-GitHub-Request-Id: C3C0:21B5F9:3BC421:4EB4D4:663A4CDB
Accept-Ranges: bytes
Date: Tue, 07 May 2024 15:46:35 GMT
Via: 1.1 varnish
X-Served-By: cache-hyd1100020-HYD
X-Cache: MISS
X-Cache-Hits: 0
X-Timer: S1715096795.188757,VS0,VE708
Vary: Authorization,Accept-Encoding,Origin
Access-Control-Allow-Origin: *
Cross-Origin-Resource-Policy: cross-origin
X-Fastly-Request-ID: b0c5c15fee71f94913e773dadca37a713e936d69
Expires: Tue, 07 May 2024 15:51:35 GMT
Source-Age: 0

]
